# Enzymes

In [2]:
import pandas as pd
import numpy as np
import re

#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense

#from pycaret.classification import *

### Remettre tout en ordre

In [6]:
""" Ouverture du document """

df = pd.read_csv('dataset_enzymes.csv', encoding='ISO-8859-1', delimiter=';')
print(df.head())
df = df.drop(['Unnamed: 8'], axis=1) # Cette ligne contient des annotations inutiles
df = df.drop(['STOP'], axis=1) # On enlève la ligne de STOP
df = df.drop([0, 1, 2, 5, 7]) # Il manque trop d'infos ou lignes vides

for i in range(7):
    df = df.drop([f'Unnamed: {i}'], axis=1) # Que des NaN
    
    
""" Rename les colonnes et les lignes """

for i in range(13,31):
    df = df.rename(columns={f'Unnamed: {i}': df[f'Unnamed: {i}'][6]})
for j in [7, 9, 10, 11, 12]:
    df = df.rename(columns={f'Unnamed: {j}': df[f'Unnamed: {j}'][14]})
    
df = df.rename(index={3: 'Nom', 4: 'Pureté', 6:'Abréviation'})
for k in range(8, 13):
    df = df.rename(index={k: df['Acc. Number'][k]})
for h in range(16, 468):
    df = df.rename(index={h: df['Acc. Number'][h]})

for l in range(468, 904):
    df = df.drop([l])
df = df.drop(['Acc. Number'], axis=1) # On a déjà rentré les infos
df = df.drop([13, 14, 15]) # On a déjà rentré les infos
 

""" Enlever les colonnes inutiles """
    
    # On enlève la ligne qu'on ne connait pas 
df = df.drop(['Q5N2G0'])

# S'il y a - et -, on enlève la ligne
for nom in df.index:
    if df['Induction (Gel)'][nom] == '-' and df['Expression (Gel)'][nom] == '-':
        df = df.drop([nom])

# Maintenant on enlève le reste des données
#df = df.drop(df.columns[:4], axis=1)
#df = df.drop(df.index[:8])
        
name_lines = df.index
name_columns = df.columns

# Si l'une des valeurs est négative ou nan, on remplace par 0
for i in range(len(name_columns)):
    for j in range(len(name_lines)):
        cell = df[name_columns[i]][name_lines[j]]
        # On remplace les , par des .
        cell = str(cell).replace(',', '.')
        try:
            if float(cell) < 0:
                cell = 0
        except:
            pass

# On stocke le bradford
bradford = {}
for line in range(8, len(df.index)):
    bradford[df.index[line]] = float(df['Bradford (mg/mL)'][line].replace(',', '.'))

# Puis on enlève les colonnes inutilisables
df = df.drop(['TAM Ref.', 'Bradford (mg/mL)', 'Induction (Gel)', 'Expression (Gel)'], axis=1)
    
# Et les puretés
purities = {}
for row in range(len(df.columns)):
    if df[df.columns[row]]['Pureté'] !=  '---':
        purities[df.columns[row]] = int(df[df.columns[row]]['Pureté'][:-1])
    else:
        purities[df.columns[row]] = 100

# Et les lignes
for line in range(8):
    df = df.drop([df.index[0]])
        
df.head(10)

   Unnamed: 0  Unnamed: 1  Unnamed: 2  Unnamed: 3  Unnamed: 4 Unnamed: 5  \
0         NaN         NaN         NaN         NaN         NaN        NaN   
1         NaN         NaN         NaN         NaN         NaN        NaN   
2         NaN         NaN         NaN         NaN         NaN        NaN   
3         NaN         NaN         NaN         NaN         NaN        NaN   
4         NaN         NaN         NaN         NaN         NaN        NaN   

                                          Unnamed: 6 Unnamed: 7 Unnamed: 8  \
0  Tableau récapitulatif des résultats obtenus lo...        NaN        NaN   
1                                                NaN        NaN        NaN   
2                                                NaN        NaN        NaN   
3  Nom des substrats accepteurs (nomenclature IUPAC)        NaN        NaN   
4                                             Pureté        NaN        NaN   

  Unnamed: 9  ...                    Unnamed: 22  \
0        NaN  ...     

,2OATA,Pyr,2OPAA,2OHAA,4MT2OB,MOPA,tMePyr,2O2PAA,2O4PBA,HPA,3H2OBA,BOHiPr,BOHPh,4H2OA,4H4PA,2KDGA,"3,3diMe",NoCeto
C7RAM1,"0,80","0,70","0,03","0,10","0,12","0,02","0,02","0,01","0,06","0,00","0,02","0,01","0,01","0,09","0,36","0,01","0,01","0,01"
P44336,"2,72","0,52","0,13","0,10","0,35","0,03","0,01","0,01","0,07","-0,01","0,03","0,01","0,01","0,14","0,26","0,01","0,01","0,01"
Q7VLP0,"1,10","0,91","0,24","0,21","0,39","0,06","0,01","0,01","0,08","0,00","0,03","0,02","0,02","0,19","0,54","0,01","0,02","0,01"
Q5QZ48,"0,55","0,24","0,11","0,07","0,16","0,03","0,01","0,01","0,08","0,00","0,02","0,01","0,01","0,08","0,20","0,01","0,01","0,01"
E1STN2,"0,11","0,07","0,02","0,06","0,12","0,02","0,01","0,02","0,06","0,00","0,01","0,01","0,01","0,04","0,06","0,01","0,01","0,01"
C4LEG2,"1,33","0,19","0,08","0,12","0,18","0,05","0,02","0,01","0,06","0,03","0,02","0,02","0,02","0,09","0,11","0,01","0,01","0,02"
Q15T03,"0,05","0,05","0,01","0,05","0,09","0,01","0,01","0,01","0,05","0,00","0,00","0,01","0,01","0,02","0,05","0,00","0,01","0,01"
Q482G6,"0,03","0,05","0,02","0,06","0,10","0,01","0,01","0,01","0,05","0,00","0,01","0,01","0,01","0,03","0,05","0,01","0,01","0,01"
Q3ILA3,"0,53","0,15","0,07","0,08","0,15","0,03","0,01","0,01","0,07","0,00","0,01","0,01","0,01","0,07","0,12","0,01","0,01","0,01"
Q12MU6,"0,13","0,08","0,02","0,06","0,10","0,02","0,01","0,01","0,05","0,00","0,01","0,01","0,01","0,03","0,06","0,01","0,01","0,01"


### Ajout des séquences

In [3]:
dict_sequences = {}

with open('FASTAs_TAMA.fasta', 'r') as sequences:
    lines = sequences.readlines()
    for j in range(len(lines)):
        if j%2 == 0: # Si c'est la ligne header
            code = re.search(r'\|([A-Z0-9]+)\|', lines[j]).group(1) # On extrait le code de l'enzyme
            if code in name_lines:
                dict_sequences[code] = ''
        else: # Si c'est la ligne avec le code protéique
            if code in name_lines:
                dict_sequences[code] = lines[j].strip() # On l'ajoute au dictionaire

print(dict_sequences)

{'A5FJ40': 'MNHILSDRINNLATSQTLAMAALARELKAQGKDIISLSLGEPDFNTPDFIKEAAKKAIDDNYSTYSPVDGYQDLKEAICRKFKRDNDLEYKPSQIVVSTGAKQSLYNIAQVMLNDGDEVILPAPYWVSYFEIVKLSGGVPVEVPTSVETDFKITPEQLEAAITPKTKMMWFSSPCNPSGSVYSREELTALAKVLEKHPNIYVVADEIYEHINFSGTFCSIGSIPGMLEKTITVNGVAKAFAMTGYRIGYIGAPEFIAKACTKIQGQVTSGANSVAQRATITAVDADPSVLNEMVQAFHGRRDLVVGLLKEIPGVKINVPEGAFYVFPDVSSFFGKTLKGHEIKDANDVSMYLLAEANVATVTGDAFGNPNCIRFSYATSNDILKEALRRIKEALTA', 'A6LJ87': 'MIKKNYLLAPGPTPVPFDILLEGARETIHHRTPQFVSILEETLNELKYLFQTENKVYTLLSSGTGALEAAITNLLNPGDKAIIVEAGKFGERWREIAERFNINVVSIKLEWGEAVTPEQIKEAIEKHPDAKAVFTTYSETSTGTVIDLEGIAKVTRNTDVVLVTDAVSALLAEPLKMDEWGIDVVVSGSQKGVMLPPGLAFIALNDKAWKLVEKSNNSSYYFNLKAYAKKYPDNPWTPGVNLIYMLRKAIKIIKEEGIENVWERHRILADATRAAVNAMGLELFSKRPGNVATAVKVPEGIDGNKLTKIMRDKYGVTIAGGQEHLKGKIFRISTLGYLSIFDTIVGISALEFTLNELGYKVEFGTGIKAAQEVLFKEVNK', 'A6US03': 'MKQMDTEKLLMIPGPTVVPTRVLNVMALPIIGHRTKDFGELTEDTVDKMKQVFQTKNESYIVTGSGTAVMDMAIANTVDKGDKVLNIVNGAFGDRFYKISSVYKADAIKLENEWGDLADPNAVKEILDENEDIKAVTIVHNETSSGAKNPIEEIGNVVKDYNALYIVDTISSLGGDYVHVDKFN

### Normaliser avec Bradford

In [4]:
for line in range(len(df.index)):
    for row in range(len(df.columns)):
        n_line = df.index[line]
        n_row = df.columns[row]
        cell = df[n_row][n_line]
        df[n_row][n_line] = float(cell.replace(',', '.')) / bradford[n_line] # On normalise

df.head(10)

,2OATA,Pyr,2OPAA,2OHAA,4MT2OB,MOPA,tMePyr,2O2PAA,2O4PBA,HPA,3H2OBA,BOHiPr,BOHPh,4H2OA,4H4PA,2KDGA,"3,3diMe",NoCeto
C7RAM1,0.276434,0.24188,0.010366,0.034554,0.041465,0.006911,0.006911,0.003455,0.020733,0.0,0.006911,0.003455,0.003455,0.031099,0.124395,0.003455,0.003455,0.003455
P44336,1.222472,0.233708,0.058427,0.044944,0.157303,0.013483,0.004494,0.004494,0.031461,-0.004494,0.013483,0.004494,0.004494,0.062921,0.116854,0.004494,0.004494,0.004494
Q7VLP0,0.773015,0.639494,0.168658,0.147576,0.274069,0.042164,0.007027,0.007027,0.056219,0.0,0.021082,0.014055,0.014055,0.133521,0.37948,0.007027,0.014055,0.007027
Q5QZ48,0.19685,0.085898,0.03937,0.025054,0.057266,0.010737,0.003579,0.003579,0.028633,0.0,0.007158,0.003579,0.003579,0.028633,0.071582,0.003579,0.003579,0.003579
E1STN2,0.02931,0.018652,0.005329,0.015987,0.031974,0.005329,0.002665,0.005329,0.015987,0.0,0.002665,0.002665,0.002665,0.010658,0.015987,0.002665,0.002665,0.002665
C4LEG2,0.203738,0.029105,0.012255,0.018382,0.027574,0.007659,0.003064,0.001532,0.009191,0.004596,0.003064,0.003064,0.003064,0.013787,0.01685,0.001532,0.001532,0.003064
Q15T03,0.015083,0.015083,0.003017,0.015083,0.027149,0.003017,0.003017,0.003017,0.015083,0.0,0.0,0.003017,0.003017,0.006033,0.015083,0.0,0.003017,0.003017
Q482G6,0.010263,0.017106,0.006842,0.020527,0.034211,0.003421,0.003421,0.003421,0.017106,0.0,0.003421,0.003421,0.003421,0.010263,0.017106,0.003421,0.003421,0.003421
Q3ILA3,0.091285,0.025835,0.012056,0.013779,0.025835,0.005167,0.001722,0.001722,0.012056,0.0,0.001722,0.001722,0.001722,0.012056,0.020668,0.001722,0.001722,0.001722
Q12MU6,0.051896,0.031936,0.007984,0.023952,0.03992,0.007984,0.003992,0.003992,0.01996,0.0,0.003992,0.003992,0.003992,0.011976,0.023952,0.003992,0.003992,0.003992


### Enlever les lignes inutiles du fichier FASTA final

In [5]:
# On enlève les enzymes dont on s'occupe pas)

with open('FASTAs_TAMA.fasta', 'r') as sequences:
    with open('FASTAs_Final.fasta', 'w') as final_sequences: # Fichier final
        lines = sequences.readlines()
        for j in range(len(lines)):
            if j%2 == 0:
                code = re.search(r'\|([A-Z0-9]+)\|', lines[j]).group(1)
                if code in name_lines: # On ajoute la ligne au fichier comme dans l'original
                    final_sequences.write(lines[j])
                    final_sequences.write(lines[j+1])

### Alignement des données

In [6]:
# Pour pas perdre de temps à rerun à chaque fois
import json
change_final_dict = False # Est-ce qu'on veut run à nouveau l'executable ou juste load le dictionnaire sauvegardé

def get_aligned_sequences():
    if change_final_dict == False:
        with open("save.json", "r") as json_file:
            dict_aligned_seq = json.load(json_file)
    return dict_aligned_seq

In [7]:
# Le format du fichier est multiline_fasta

from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio.Align.Applications import ClustalwCommandline

In [8]:
if change_final_dict == True: # Si on veut relancer l'executable pour refaire les calculs
    
    clustalw_exe = "clustalw2.exe" # L'emplacement que l'executable qui va faire le traitement des données
    output_file = "output.aln"
    input_file = "FASTAs_Final.fasta"

    sequences = list(SeqIO.parse(input_file, "fasta"))

    seq_records = [SeqRecord(seq.seq, id=seq.id) for seq in sequences] # Créer un objet SeqRecord pour chaque séquence

    temp_file = "temp.fasta" # Créer un fichier temporaire pour stocker les séquences à aligner
    SeqIO.write(seq_records, temp_file, "fasta")

    clustalw_cline = ClustalwCommandline( # Initialiser l'objet ClustalwCommandline pour traiter correctement les données
        cmd=clustalw_exe,
        infile=input_file,
        outfile=output_file,
        output="clustal",  # Format de sortie
        align=True,  # Effectuer un alignement complet
    )

    clustalw_cline() # Exécution de la commande ClustalW2

    aligned_sequences = list(SeqIO.parse(output_file, "clustal")) # Maintenant on peut lire les séquences alignées

    for aligned_seq in aligned_sequences:
        # Accéder à l'identifiant de la séquence
        seq_id = aligned_seq.id
        print("ID de la séquence :", seq_id)

        # Accéder à la séquence elle-même
        sequence = aligned_seq.seq
        print("Séquence :", sequence)

    # On crée à nouveau un dictionnaire avec les nouvelles séquences alignées
    dict_aligned_seq = {}

    for aligned_seq in aligned_sequences:

        code = re.search(r'\|([A-Z0-9]+)\|', aligned_seq.id).group(1)
        sequence = aligned_seq.seq
        dict_aligned_seq[code] = str(sequence)

    save_file = "save.json"

    with open(save_file, 'w') as s_file:
        json.dump(dict_aligned_seq, s_file) # Sauvegarder le dictionnaire dans le fichier JSON
dict_aligned_seq = get_aligned_sequences() 
print(dict_aligned_seq)

{'C0GI63': '------------------------------------------------------------------------MSRIIYLNGKYVP--------------------EQEAKVSVFDHGFLYGDGVFEGIRAYK---G---KVFKLEEHMARLYESAKSIML--------DIPESYEEMTEATLETIRQNNLE--------------------------------------------------DAYIRIVVSRGTGDLGLDPRKCP-KATT---------VIIASSITLFPEEL-YEQGLEVITVATRRN-------VPDALDPQIKSL-NYLNNILVKIEANRAGVMEAIMLNGQGYVAEG--------TGDNVFIVKRGKILTPPSYVG-----ALCGITRQVIID-LAAEAGYLLEER-PFTRHELYVADECFLTGTAAEVIPVIGVDGRQIGS------------GTPG-----PVTRDLVARFRELAGTTGTDIHP-----------------------------------------------', 'F0T2Q0': '--------------------------------------------------------------------------MIVFCDGEFVE--------------------AEQAKISVFDHGFLYGDGVFEGIRAYN---G---RVFKLKEHIDRLYESANAILL--------KIEVSRTEMIAIVTETVRRNKLQ--------------------------------------------------DAYIRLVVSRGTGDLGLDPQKCR-RSEI---------FCIAGQITLFPQSM-YENGLEVITVSTRRN-------NPNAINPMVKSL-NYLNNILAKIECNRAGVMEAIMLNQEGYVCEG--------TGDNIFIVKDGIIKTPPTYVG-----ILKGITRDTVLD-LAGEMGIAALEQ-PFTLHELY